# **Web Scrapping Of Ambition Box Website**


# Importing Required libraries

In [ ]:
import numpy as np  # Numerical operation
import pandas as pd  # For storing and manipulating scraped data in tabular form
import re  # Regular expressions for text cleaning
import seaborn as sns   # Visualization libraries
import matplotlib.pyplot as plt 
import requests  # To send HTTP requests to the website
from bs4 import BeautifulSoup   # To parse and extract data from HTML
import warnings   # To ignore unnecessary warning messages
warnings.filterwarnings("ignore")

In [ ]:
# This page contains details of multiple companies (name, rating, reviews, etc.)
url = "https://www.ambitionbox.com/list-of-companies?"

In [ ]:
response=requests.get(url)
response

# Sending a request to the website
# <Response [403]> indicates "Forbidden" access.
# This means the website blocked the request, likely due to:
# 1. Missing request headers (User-Agent)
# 2. Anti-scraping protection

In [ ]:
# To bypass the 403 error, we add a User-Agent header
# to mimic a real browser request.

requests_header={"content-type":"text/plain",
                "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
                "accept-encoding":"gzip, deflate, br, zstd"}

In [ ]:
#  Sending HTTP GET request to AmbitionBox website
response=requests.get(url,headers=requests_header)

In [ ]:
# Check if request was successful 
response

In [ ]:
# Get the HTML content (source code) returned by the HTTP response
pagecontent=response.text

In [ ]:
# Parse the HTML content using BeautifulSoup to create a searchable DOM tree
soup=BeautifulSoup(pagecontent)

In [ ]:
# Find all company card sections using div tag and specific CSS class
companies = soup.find_all("div", class_="companyCardWrapper__companyPrimaryDetailsTopSection")
# Empty list to store extracted company names
company_names=[]
# Loop through each company card
for company in companies:
    # Extract text content from the div, remove extra spaces/newlines,
    # and add the cleaned company name to the list
    company_names.append(company.text.strip())
# Print first 10 company names to verify the output
print(company_names[0:10])

In [ ]:
# Find all rating elements using div tag and rating CSS class
ratings=soup.find_all("div", class_="rating_text")
rates=[]
for i in ratings:
    rates.append(i.text.strip())
print(rates[0:10])

In [ ]:
# List to store number of operational locations for each company
operations_spread=[]
# Find all span elements that contain location information
for i in soup.find_all("span", class_="companyCardWrapper__interLinking"):
    text = i.text.strip()
    # Use regex to find patterns like '+3 other locations'
    # It captures the number after '+'
    loc=re.findall(r"\+(\d+)\s+other\s+locations",text) 
    # If additional locations are found
    if loc:
        # Add 1 to include the main location
        operations_spread.append(int(loc[0])+1)
    else:
        # If location info is missing, store NaN
        operations_spread.append(np.nan)

print(operations_spread[0:10])

In [ ]:
# Find all blocks that contain rating comparison information
# (Highly Rated For / Critically Rated For)

rating_blocks = soup.find_all("div", class_="companyCardWrapper__ratingComparisonWrapper")
# Find all company title elements (used for alignment)
titles_on_page = soup.find_all("a", class_="companyCardWrapper__companyName")

# Lists to store extracted ratings
h_rate=[]
c_rate=[]

# Loop through each company index
for i in range(len(titles_on_page)):

    # Check if rating block exists for this company
    if i < len(rating_blocks):
        text = rating_blocks[i].text.strip()

        # Extract text after 'Highly Rated For' and before
        # 'Critically Rated For' (if present)
        match_high = re.search(r"Highly Rated For (.*?)(?:Critically Rated For|$)", text)
        h_rate.append(match_high.group(1).strip() if match_high else np.nan)

        # Extract text after 'Critically Rated For'
        match_crit = re.search(r"Critically Rated For (.*)", text)
        c_rate.append(match_crit.group(1).strip() if match_crit else np.nan)

    # If rating block is missing for the company
    else:
        h_rate.append(np.nan)
        c_rate.append(np.nan)

print(h_rate[0:10])
print("\n")
print(c_rate[0:10])

In [ ]:
# Find all span elements that contain company type information
typeofcomp=soup.find_all("span", class_="companyCardWrapper__interLinking")
# List to store extracted company types
types=[]

# Loop through each company type span
for i in typeofcomp:

    # Check if the text contains '|' separator
    # (AmbitionBox uses 'Type | Location | Size' format)
    if "|" in str(i):
        # Split text at '|' and take the first part as company type
        text = i.text.split("|")[0].strip()
        types.append(text)
    else:
        # If type information is missing, assign NA
        types.append("NA")

print(types[0:10])

In [ ]:
locations=soup.find("span", class_="companyCardWrapper__interLinking")
locations

In [ ]:
# Find all span elements that contain combined company info
# (Type | Location | Size | etc.)

locations=soup.find_all("span", class_="companyCardWrapper__interLinking")
location=[]
# Loop through each span element
for i in locations:
    # Check if the text contains '|' separator
    if "|" in str(i):
        # Split by '|', take the SECOND part (location),
        # remove extra spaces, then take only the first word
        text = i.text.split("|")[1].strip().split(" ")[0]
        location.append(text)
    else:
         # If location info is missing
        location.append("NA")

print(location[0:20])

In [ ]:
counts=soup.find("span", class_="companyCardWrapper__ActionCount")
counts

In [ ]:
# Find all span elements that contain action counts
# (Reviews, Salaries, Interviews, Jobs, Benefits, Photos)

counts=soup.find_all("span", class_="companyCardWrapper__ActionCount")

# Lists to store extracted values
reviews=[]
reviews_1=[]
salaries=[]
interviews=[]
jobs=[]
benefits=[]
photos=[]
for i in counts:
    # Append raw text (count) to reviews list
    reviews.append(i.text)
    # From the combined list, extract values using step slicing
    # Every 6 values correspond to one company:
    # index 0 → Reviews
    # index 1 → Salaries
    # index 2 → Interviews
    # index 3 → Jobs
    # index 4 → Benefits
    # index 5 → Photos
    reviews_1.extend(reviews[0::6])
    salaries.extend(reviews[1::6])
    interviews.extend(reviews[2::6])
    jobs.extend(reviews[3::6])
    benefits.extend(reviews[4::6])
    photos.extend(reviews[5::6])
print(reviews[0:10])
print("\n")
print(reviews_1[0:10])
print("\n")
print(salaries[0:10])
print("\n") 
print(interviews[0:10])
print("\n")
print(jobs[0:10])
print("\n")
print(benefits[0:10])
print("\n")
print(photos[0:10])

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
company_names=[]
rates=[]
rate_count = []
types=[]
location=[]
reviews_1=[]
salaries = []
interviews =[]
jobs = []
benefits = []
photos = []
h_rate=[]
c_rate=[]
operations_spread=[]
us="https://www.ambitionbox.com/list-of-companies?"
for i in range(1,26):
    url=us+"page="+str(i)
    page = requests.get(url, headers=requests_header)
    soup=BeautifulSoup(page.text)
    companies = soup.find_all("div", class_="companyCardWrapper__companyPrimaryDetailsTopSection")
    for company in companies:
        company_names.append(re.findall(r"\w+",company.text))
        
    ratings=soup.find_all("div", class_="rating_text")
    for i in ratings:
        rates.append(re.findall(r"\w+\W\w+",i.text))
    
     # operations_spread
    for i in soup.find_all("span", class_="companyCardWrapper__interLinking"):
        text = i.text.strip()
        loc=re.findall(r"\+(\d+)\s+other\s+locations",text) 
        if loc:
            operations_spread.append(int(loc[0])+1)
        else:
            operations_spread.append(np.nan)

    # Highly Rated & Critically rated
    rating_blocks = soup.find_all("div", class_="companyCardWrapper__ratingComparisonWrapper")
    titles_on_page = soup.find_all("a", class_="companyCardWrapper__companyName")
    
    for i in range(len(titles_on_page)):
        if i < len(rating_blocks):
            text = rating_blocks[i].text.strip()
    
            match_high = re.search(r"Highly Rated For (.*?)(?:Critically Rated For|$)", text)
            h_rate.append(match_high.group(1).strip() if match_high else np.nan)
    
            match_crit = re.search(r"Critically Rated For (.*)", text)
            c_rate.append(match_crit.group(1).strip() if match_crit else np.nan)
        else:
            h_rate.append(np.nan)
            c_rate.append(np.nan)

    typeofcomp=soup.find_all("span", class_="companyCardWrapper__interLinking")
    for i in typeofcomp:
        if "|" in str(i):
            text = i.text.split("|")[0].strip()
            types.append(text)
        else:
            types.append("NA")
        
    locations=soup.find_all("span", class_="companyCardWrapper__interLinking")
    for i in locations:
        if "|" in str(i):
            text = i.text.split("|")[1].strip().split(" ")[0]
            location.append(text)
        else:
            location.append("NA")
    counts=soup.find_all("span", class_="companyCardWrapper__ActionCount")
    reviews=[]
    for i in counts:
        reviews.append(i.text)
    reviews_1.extend(reviews[0::6])
    salaries.extend(reviews[1::6])
    interviews.extend(reviews[2::6])
    jobs.extend(reviews[3::6])
    benefits.extend(reviews[4::6])
    photos.extend(reviews[5::6])
    print(url)

In [ ]:
print(len(company_names))
print(len(reviews_1))
print(len(salaries))
print(len(location))


In [ ]:
d={
    "Company":company_names,
    "Rating":rates,
    "Industry":types,
    "Location":location,
    "Reviews":reviews_1,
    "Salaries":salaries,
    "Interviews":interviews,
    "Jobs":jobs,
    "Benefits":benefits,
    "Photos":photos,
    "Highly_Rated_For":h_rate,
    "Critically_Rated_For":c_rate,
    "Total_Operations_Spread":operations_spread}

In [ ]:
df = pd.DataFrame(d)
df

In [ ]:
raw_df = df.copy()

# Save raw data
raw_df.to_csv("ambitionbox_raw_data.csv", index=False)

print("Raw data saved successfully.")


In [ ]:
df=pd.read_csv("ambitionbox_raw_data.csv")

In [ ]:
df

# Initial Data Inspection 

In [ ]:
# shape of Dataframe
df.shape

In [ ]:
df.info()

In [ ]:
df.describe(include = 'all')

- **Data Quality Check**
# Identify Null Values 

In [ ]:
df.isnull().sum()

In [ ]:
# to see null value percentage
(df.isnull().sum() / len(df)) * 100


# Identify Duplicate Values 

In [ ]:
df.duplicated().sum()

# Outlier Detection

In [ ]:
num_cols = df.select_dtypes(include = ["int64","float64"]).columns
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR
    
    outliers = df[(df[col] < lower_limit) | (df[col] > upper_limit)]
    
    print(f"{col} Outliers:", len(outliers))


# Visual of Outliers 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for col in num_cols:
    plt.figure(figsize=(5,3))
    sns.boxplot(x=df[col])
    plt.title(col)
    plt.show()


# Unique Value Analysis

In [ ]:
for col in df.columns:
    print(f"\nColumn: {col}")
    print("Unique Count:", df[col].nunique())
    print(df[col].unique()[:5])
    print("-"*30)

In [ ]:
for col in df.columns:
    print(f"\nColumn: {col}")
    print("Unique Count:", df[col].nunique())
    print(df[col].unique())
    print("-"*30)

# Comapny Name Cleaning ( Removing "," spaces,in between the names)

In [ ]:
import ast

col = "Company"

def clean_company(x):
    try:
        # Convert string representation of list into actual list
        company_list = ast.literal_eval(x)
        
        # Join words with space (removes commas automatically)
        return " ".join(company_list)
    
    except:
        return x

df[col] = df[col].apply(clean_company)

# Optional: remove extra spaces
df[col] = df[col].str.replace(r"\s+", " ", regex=True).str.strip()


In [ ]:
import ast
import re

col = "Company"

def clean_company(x):
    try:
        company_list = ast.literal_eval(x)
        name = " ".join(company_list)
    except:
        name = x

    # Fix L T → L&T
    name = re.sub(r"\bL T\b", "L&T", name)

    # Fix R R → R.R.
    name = re.sub(r"\bR R\b", "R.R.", name)

    # Fix BYJU S → BYJU'S
    name = re.sub(r"\bBYJU S\b", "BYJU'S", name)

    # Remove extra spaces
    name = re.sub(r"\s+", " ", name).strip()

    return name

df[col] = df[col].apply(clean_company)


In [ ]:
import re

col = "Company"

# Create correction dictionary
corrections = {
    "Dr Reddy s": "Dr. Reddy's",
    "Domino s Pizza": "Domino's Pizza",
    "McDonald s": "McDonald's",
    "S P Global": "S&P Global",
    "Automatic Data Processing ADP": "Automatic Data Processing (ADP)",
    "Paisabazaar com": "Paisabazaar.com",
    "vivo": "Vivo",
    "Whitehat jr": "WhiteHat Jr",
    "Exl India": "EXL India",
    "24 7 ai": "24/7.ai",
    "Statestreet HCL Services": "State Street HCL Services",
    "Star Union Dai ichi Life Insurance": "Star Union Dai-ichi Life Insurance",
    "JPMorgan Chase Co": "JPMorgan Chase & Co.",
    "Ernst Young": "Ernst & Young",
    "Mahindra Mahindra": "Mahindra & Mahindra",
    "Larsen Toubro Limited": "Larsen & Toubro Limited"
}

# Apply corrections
df[col] = df[col].replace(corrections)

# Optional: strip extra spaces again
df[col] = df[col].str.strip()


In [ ]:
print(df[col].unique())

# Identify Brackets Issue
-  if result > 0 → brackets problem exists

In [ ]:
df['Company'].str.contains(r"\[|\]").sum()

# Identify k / L Salary Issue
- Salary might look like:
3.5k, 
2.1L, 
12k

In [ ]:
df['Salaries'].unique()[:10]

In [ ]:
df['Salaries'].str.contains('k', case=False, na=False).sum()
df['Salaries'].str.contains('l', case=False, na=False).sum()


# Identify Inconsistent Data Types

In [ ]:
df.dtypes

In [ ]:
df.head(1)

# **DATA CLEANING** 

In [ ]:
df["Company"] = (df["Company"].astype(str).str.replace("[", "", regex=False)
    .str.replace("]", "", regex=False).str.replace("'", "", regex=False)
    .str.strip())

In [ ]:
df["Rating"] = df["Rating"].str.strip("[]'")

# Fix k/L Salary Issue
Convert:
- 5k → 5000
- 3.2L → 320000

In [ ]:
def convert_to_number(x):
    if pd.isna(x):
        return None
    x = str(x).lower().replace(",", "")
    if "l" in x: 
        return float(x.replace("l", "")) * 100000
    elif "k" in x:  
        return float(x.replace("k", "")) * 1000
    else:
        try:
            return float(x)
        except:
            return None
            
cols_to_convert = ["Salaries", "Rating", "Reviews", "Interviews", "Jobs", "Benefits", "Photos"]
for col in cols_to_convert:
    df[col] = df[col].apply(convert_to_number)

In [ ]:
df.info()

In [ ]:
df.head(3)

# **Handle Missing Values** 

In [ ]:
df.isnull().sum()     
# Post-Conversion Check
# After converting into numbers , Invalid values like "" "NA" will became NAN

In [ ]:
df["Industry"].fillna("Unknown", inplace=True)
df["Location"].fillna("Unknown", inplace=True)
df["Highly_Rated_For"].fillna("Not Mentioned", inplace=True)
df["Critically_Rated_For"].fillna("Not Mentioned", inplace=True)

In [ ]:
df['Jobs'] = df['Jobs'].fillna(0)
df['Photos']= df['Photos'].fillna(0)
# (if missing means no jobs/photos

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

# **Removing Duplicates**

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

# **Outlier Treatment**

In [ ]:
num_cols = df.select_dtypes(include=['float64','int64']).columns
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower) | (df[col] > upper)]
    print(f"{col} Outliers:", len(outliers))


# **Capping Outliers**

for col in num_cols:
    df[col] = np.where(df[col] > upper, upper,
                   np.where(df[col] < lower, lower, df[col]))
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    print(f"{col} Outliers:", len(outliers))

In [ ]:
# Final validation
df.describe()

In [ ]:
clean_df = df.copy()
# Save cleaned data
clean_df.to_csv("ambitionbox_cleaned_data.csv", index=False)

print("Cleaned data saved successfully.")


In [ ]:
df.columns

# **Exploratory Data Analysis (EDA)**

# Dataset Overview
- The dataset contains 500 companies and 13 variables.
- Numeric variables include Ratings, Salaries, Reviews, Interviews, Jobs,    Benefits, Photos, and Operations Spread.
- Categorical variables include Industry, Location, Highly Rated For, and Critically Rated For.
- Ratings range between approximately 2.5 and 4.8.
- Salary distribution shows high variation across companies.

# Visualizations After cleaning the data

# **"Univariate Analysis" for  Numeric Variables**

In [ ]:
# Histogram for Ratings column

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6,4))
sns.histplot(df['Rating'].dropna(), bins=20, kde=True, color='skyblue')

plt.title("Distribution of Ratings", fontsize=12, fontweight='bold')
plt.xlabel("Ratings")
plt.ylabel("Frequency")
plt.show()

# Observations – Distribution of Ratings

- The ratings are mostly concentrated between 3.5 and 4.2, indicating generally positive feedback.
- The peak frequency is around 3.8–4.0, showing that most companies have ratings close to 4.
- Very few ratings are below 3.0 or above 4.5, meaning extreme low or high ratings are rare.

- The distribution appears approximately normal (bell-shaped) with slight skewness.

- Overall, the dataset suggests that company ratings are moderately high and consistent, with no major outliers.

- This indicates generally positive employee satisfaction.

In [ ]:
# Boxplot of Salaries

plt.figure(figsize=(6,4))
sns.boxplot(x=df['Salaries'].dropna(), color='salmon')

plt.title("Boxplot of Salaries", fontsize=12, fontweight='bold')
plt.xlabel("Salaries")
plt.show()

# Observations – Boxplot of Salaries

- The salary distribution is highly right-skewed, as many outliers are present on the higher end.

- Most salary values are concentrated in a lower range (compact box near the left side).

- The median salary lies close to the lower quartile, indicating majority of employees earn within a limited lower band.

- There are several extreme high-value outliers, showing a small group earning significantly higher salaries.

- This suggests salary inequality, where few individuals earn much more compared to the majority

In [ ]:
# Violin Plot Reviews

plt.figure(figsize=(6,4))
sns.violinplot(x=df['Reviews'].dropna(), color='lightcoral')

plt.title("Violin Plot of Reviews", fontsize=12, fontweight='bold')
plt.xlabel("Reviews")
plt.show()

# Observation - Violin Plot of Reviews

- The distribution of reviews is highly right-skewed, with a long tail extending toward higher values.

- Most companies have low to moderate number of reviews, concentrated in the lower range.

- A small number of companies have very high review counts (extreme values).

- The median lies close to the lower end, indicating that majority receive fewer reviews.

- The wide spread on the right side shows large variation in review counts across companies.

In [ ]:
# KDE plot of Jobs

plt.figure(figsize=(6,4))
sns.kdeplot(df['Jobs'].dropna(), fill=True, color='orange')

plt.title("KDE Plot of Jobs", fontsize=12, fontweight='bold')
plt.xlabel("Jobs")
plt.ylabel("Density")
plt.show()

# Observation - 

- The distribution of jobs is highly right-skewed, with a sharp peak at lower values.

- Most data points are concentrated in the lower job count range.

- There is a long tail extending toward higher values, indicating a few companies have very high job openings.

- The density quickly decreases as job count increases, showing high variation across companies.

- Overall, majority of companies post limited job openings, while only a few dominate with large numbers.

# **Univariate Analysis for Categorical Columns**

# Barplot for Industry Column

In [ ]:
plt.figure(figsize=(5,3))
df["Industry"].value_counts(dropna=False).head(10).plot(kind='bar', color='salmon')
plt.title(f"Top 10 Categories in industry",color='black')
plt.ylabel("Count")
plt.xticks(rotation=90, ha='right')
plt.tight_layout(rect=[0, 1, 1, 0.97])
plt.show(); 

# Observations

- IT Services & Consulting has the highest count, clearly dominating other industries.

- Pharma, Insurance, and Banking follow as the next major sectors.

- The remaining industries (Internet, BPO, Financial Services, NBFC, Automobile, Engineering & Construction) show moderate and relatively similar counts.

- The distribution indicates that the dataset is heavily focused on IT-related companies.

- There is a noticeable gap between the top industry and the others, showing industry imbalance in the dataset.

In [ ]:
plt.figure(figsize=(5,5))
df['Location'].value_counts().head(5).plot(kind='pie', autopct='%1.1f%%', startangle=90, cmap='Pastel1')
plt.title("Distribution of Companies by Location",color='black')
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show();

# Observations - Distribution of Companies by Location

- Bengaluru has the highest share (31.5%), making it the leading business hub in the dataset.
- Mumbai follows closely with 28.7%, showing strong corporate presence.
- Pune accounts for 15.4%, representing moderate company concentration.
- Chennai (12.6%) and Hyderabad (11.8%) have comparatively lower representation.
- The majority of companies are concentrated in Bengaluru and Mumbai, indicating these cities dominate the dataset.

In [ ]:
plt.figure(figsize=(4,3))
df["Critically_Rated_For"].value_counts().head(10).plot(kind="barh")
plt.title("Most Common critically rates")
plt.tight_layout(rect=[0, 0, 1, 0.97])

plt.show(); 

# Observation 

- “Promotions / Appraisal” is the most frequently criticized aspect.
- A large number of companies are negatively rated for promotion and appraisal policies.
- “Unknown” also appears significantly (companies with no specified criticism).
- Other recurring issues include:
- Salary & Benefits
- Work Satisfaction
- Company Culture
- Job Security
- Work-Life Balance

# Bivariate Analysis

## categorical vs numerical

In [ ]:
top_reviewed_companies = df[['Company', 'Reviews']].sort_values(by='Reviews', ascending=False).head(10)
plt.figure(figsize=(6,4))
sns.barplot(x='Reviews', y='Company', data=top_reviewed_companies, palette='Set2')
plt.title("Top 10 companies by total positive reviews",color="black")
plt.ylabel("company", fontsize=12)
plt.xlabel("reviews", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout(rect=[0, 0, 1, 0.97])

plt.show();  

# Observation - Top 10 Companies by Total Reviews

- TCS has the highest number of positive reviews, significantly ahead of others.

- Accenture and Wipro follow as the next most reviewed companies.

- Most top companies belong to the IT and Banking sectors, indicating strong presence in the dataset.

- There is a gradual decline in review counts from rank 1 to rank 10.

- The large difference between TCS and others suggests higher employee engagement or larger workforce size.

In [ ]:
top10_types = df.groupby('Industry')['Reviews'].mean().sort_values(ascending=False).head(10).index
top10_df = df[df['Industry'].isin(top10_types)]
sns.boxplot(data=top10_df, x="Industry", y="Reviews", palette="Set2")
plt.title("Box Plot — Reviews by Top 10 Company Types", fontsize=11)
plt.xticks(rotation=75)
plt.tight_layout(rect=[0, 0, 1, 0.97])

plt.show(); 

# Observation - Reviews by Top 10 Company Types

- IT Services & Consulting shows the highest variability with multiple extreme outliers, indicating some companies have very high review counts.

- Banking and Management Consulting also display higher median review values compared to other industries.

- Industries like Oil & Gas, Iron & Steel, and Recruitment have comparatively lower median reviews and smaller spread.

- The presence of many outliers suggests large differences in company size or popularity within the same industry.

- Overall, review distribution varies significantly across industries, with IT-related sectors dominating in review volume.

In [ ]:
a = df.groupby('Location')['Interviews'].sum().sort_values(ascending=False).head(10).reset_index()

sns.barplot(x='Location', y='Interviews', data=a, palette='Set1')

plt.title("Top 10 Locations with Highest Number of Interviews")
plt.xlabel("Location")
plt.ylabel("Total Interviews")
plt.xticks(rotation=45, ha='right') 
plt.tight_layout(rect=[0, 0, 1, 0.97])

plt.show();

# Observation 

- Bengaluru has highest interviews (~42k).
- Mumbai second (~28k).
- Pune third.
- Clear decreasing pattern.
- Metro cities dominate hiring activity.
- Bengaluru is the major hiring hub.
- Hiring activity decreases gradually after top 3 cities.
- Tier-2 cities have significantly fewer interviews.

In [ ]:
top_interviews = (
    df.sort_values(by="Interviews", ascending=False)[["Company", "Interviews"]]
      .head(10)
)
sns.barplot(
    x="Company",
    y="Interviews",
    data=top_interviews,
    palette="Set2"
)
plt.title("Top 10 Companies with Highest Interviews")
plt.xlabel("Company")
plt.ylabel("Number of Interviews")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Observation

   . The chart shows that a small group of companies accounts for the majority of interview opportunities, indicating concentrated hiring activity among top recruiters.
   
   . Bengaluru has the highest number of interviews, showing it is the top hiring hub.

   . Mumbai and Hyderabad follow, indicating strong job activity in these cities.

   . There is a steady decline from the top cities to the lower ones.

   . Pune and Chennai have moderate interview volumes.

   . Gurugram, Noida, and New Delhi show lower but noticeable activity.

   . Kolkata and Ahmedabad have the least interview counts among the top 10

## Numerical vs Numerical 

In [ ]:
plt.figure(figsize=(5,4))
sns.regplot(x="Salaries", y="Reviews", data=df,scatter_kws={'alpha':0.6},
    line_kws={'color': 'red'})
plt.title("reviews vs Salaries")
plt.tight_layout(rect=[0, 0, 1, 0.94])

plt.show(); 

# Observation- (Reviews vs Salaries)

- The scatter plot shows a positive linear relationship between salaries and reviews.

- As salary increases, the number of reviews also tends to increase.

- The regression line with upward slope confirms a strong positive correlation.

- Higher salary companies generally have higher employee engagement or larger workforce size, leading to more reviews.

- Some variability exists, but overall the trend is clearly increasing.

In [ ]:
sns.scatterplot(
    data=df,
    x="Salaries",
    y="Rating",
    alpha=1,
    color='deeppink'
)
plt.title("Rating vs Salaries")
plt.tight_layout(rect=[0, 0, 1, 0.94])
plt.show(); 

# Observation – Rating vs Salaries (Scatter Plot)

- There is no strong linear relationship between salary and rating.

- Ratings are mostly concentrated between 3.0 and 4.2, regardless of salary level.

- Even companies with lower salaries have high ratings, and some high-salary companies have moderate ratings.

- Data points are widely scattered, indicating a weak or negligible correlation.

- This suggests that higher salary does not necessarily guarantee higher employee satisfaction (rating).

## categorical vs categorical

In [ ]:
# Filter top 15 for clarity
top_types = df['Industry'].value_counts().head(15).index
top_locs = df['Location'].value_counts().head(15).index
filtered_df = df[(df['Industry'].isin(top_types)) & (df['Location'].isin(top_locs))]

plt.figure(figsize=(12, 6))
sns.countplot(data=filtered_df, x="Industry", hue="Location", palette="Set2")
plt.title("Countplot — Company Type vs Location (Top 15 Categories)", fontsize=13, color='black')
plt.xlabel("Company Type")
plt.ylabel("Count")
plt.xticks(rotation=90)
plt.legend(title="Location", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout(rect=[0, 0, 1, 0.94])
plt.show();

# Observation

- IT Services & Consulting has the highest presence across multiple locations, especially Bengaluru and Hyderabad.

- Metro cities like Mumbai, Chennai, and Pune show strong representation across major industries.

- Some industries are concentrated in specific cities, indicating regional industry dominance.

- Overall, the dataset shows IT and Banking sectors are widely distributed, while others are location-specific.

In [ ]:
# Detailed and clear visualisation of IT and Consulting Services by Location

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(data=df[df["Industry"]=="IT Services & Consulting"], x="Location")
plt.xticks(rotation=45)
plt.title("IT Services & Consulting by Location")
plt.show()

# Observation
   . Bengaluru has the highest count, making it the primary IT hub.

   . Hyderabad and Pune follow, showing strong IT hiring activity.

   . Chennai has a moderate presence in IT roles.

   . Mumbai shows lower IT concentration compared to tech cities.

In [ ]:
# Detailed and clear visualisation of Insurance by Location

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(data=df[df["Industry"]=="Insurance"], x="Location")
plt.xticks(rotation=45)
plt.title("Insurance by Location")
plt.show()

In [ ]:
# Filter top 15 values
top_types = df['Industry'].value_counts().head(15).index
top_low = df['Critically_Rated_For'].value_counts().head(15).index
filtered_df = df[(df['Industry'].isin(top_types)) & (df['Critically_Rated_For'].isin(top_low))]

# Crosstab for heatmap
cross_tab = pd.crosstab(filtered_df['Critically_Rated_For'], filtered_df['Industry'])

plt.figure(figsize=(12, 6))
sns.heatmap(cross_tab, cmap="YlGnBu", annot=True, fmt="d")
plt.title("Heatmap — Industry Type vs Critically Rated For (Top 15)", fontsize=13, color='darkblue')
plt.xlabel("Critically Rated For")
plt.ylabel("Industry Type")
plt.tight_layout(rect=[0, 0, 1, 0.94])
plt.show();

# Observation

- Promotions/Appraisal and Salary & Benefits appear frequently across many industries.

- IT Services & Consulting shows higher intensity values, indicating more frequent critical feedback.

- Certain industry–rating combinations have very low or zero counts, showing limited overlap.

- The heatmap highlights that employee concerns are more concentrated in growth and compensation-related factors.

### PIVOT TABLE

In [ ]:
# Create pivot (count of records)
pivot_ind_loc = pd.pivot_table(
    df,
    index=["Industry", "Location"],
    aggfunc="size"
).reset_index(name="Count")

# Sort by highest count
pivot_ind_loc = pivot_ind_loc.sort_values(by="Count", ascending=False)
pivot_ind_loc.head(10)

# Observation

- This table shows which industry is present more in which location.
   
- By counting the number of records for each industry and location, we can see where companies are concentrated.

- This analysis helps us see which industry is most active in each location.

- 28 companies in IT located in Bengaluru
- 15 insurance companies in Mumbai
- 13 banks in Mumbai
- 13 IT companies in Hyderabad

In [ ]:
top_interviews = (
    df.sort_values(by="Interviews", ascending=False)[["Company", "Interviews"]]
      .head(10)
      .reset_index(drop=True)
)
print(top_interviews)

# Multivariate Analysis

In [ ]:
corr = df[[ 'Rating','Salaries', 'Interviews', 'Jobs', 'Total_Operations_Spread']].corr()
sns.heatmap(corr, annot=True, cmap='Blues', fmt='.2f')
plt.title("Correlation Between Numeric Variables")
plt.show();

# Observation
- Salaries and Interviews (0.95) show a very strong positive correlation, indicating companies offering higher salaries tend to have more interview activity.

- Salaries and Jobs (0.50) and Interviews and Jobs (0.51) show a moderate positive relationship, suggesting hiring activity increases with salary levels.

- Rating has very weak correlation with Salaries (-0.16), Interviews (-0.11), and Jobs (-0.04), meaning salary or hiring does not strongly influence company ratings.

- Total Operations Spread has a slight positive relationship with Interviews (0.24) and Ratings (0.17), but overall impact is minimal.

- Overall, compensation and hiring metrics are strongly interconnected, while employee ratings remain largely independent.

In [ ]:
sns.pairplot(df, vars=['Rating','Reviews','Salaries','Interviews','Jobs','Benefits','Photos'],
             hue='Industry', diag_kind='kde', palette='viridis')

# Observation

- Strong positive relationships are visible between Salaries, Interviews, Reviews, and Benefits (clear upward trends in scatter plots).

- Rating shows weak or no strong linear relationship with most other numerical variables.

- Most variables (Reviews, Salaries, Interviews, Jobs, Benefits, Photos) are right-skewed, as seen in diagonal KDE plots.

- Clusters are visible for certain industries, but overlap is high, indicating industry differences are present but not sharply separated.

- Overall, hiring and compensation metrics are interconnected, while rating behaves independently.

In [ ]:
top_types = df['Location'].value_counts().head(10).index

# Create temporary series (NOT inside df)
types_filtered = df['Location'].apply(lambda x: x if x in top_types else 'Other')

sns.scatterplot(
    data=df,
    x='Salaries',
    y='Rating',
    size='Reviews',
    hue=types_filtered,   # use directly here
    alpha=0.6,
    sizes=(40,400),
    palette='Set1'
)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');


# Observations  – Bubble Plot (Salary vs Rating with Reviews & Location)

- There is no strong correlation between salary and rating (points scattered without clear trend).

- Bubble size (Reviews) increases with salary in some cases, indicating larger companies tend to offer higher salaries.

- Major cities like Bengaluru, Hyderabad, and Mumbai dominate higher salary ranges.

- High-salary companies do not necessarily have higher ratings.

- Review count varies significantly across locations.

# **Analytical Questions & Insights**

# Q1. Which Company Has the Highest Rating?

In [ ]:
df.sort_values(by="Rating", ascending=False)[["Company","Rating"]].head(1)

# Q2. Top 5 Highest Rated Companies

In [ ]:
df.sort_values(by="Rating", ascending=False)[["Company","Rating"]].head(5)

# Q3. Which Company Has the Most Reviews?

In [ ]:
df.sort_values(by="Reviews", ascending=False)[["Company","Reviews"]].head(1)

# Q4. Which Location Has the Highest Hiring Activity?

In [ ]:
df.groupby("Location")["Interviews"].sum().sort_values(ascending=False).head(1)

# Q5. Which Industry Has Highest Average Rating?

In [ ]:
df.groupby("Industry")["Rating"].mean().sort_values(ascending=False).head(5)

# Q6. Which Industry Has Highest Average Salary?

In [ ]:
df.groupby("Industry")["Salaries"].mean().sort_values(ascending=False).head(5)

# Q7. Is Salary Strongly Related to Rating?

In [ ]:
df[["Salaries","Rating"]].corr()

# Insight:
- Correlation value is low.
- Salary is not strong predictor of satisfaction.

# Q8. Which Company Has Maximum Operations Spread?

In [ ]:
df.sort_values(by="Total_Operations_Spread", ascending=False)[["Company","Total_Operations_Spread"]].head(1)

# Q9. Which Industry Receives Most Criticism?

In [ ]:
df["Critically_Rated_For"].value_counts().head(5)

# Q10. Top 10 Companies with Most Job Openings

In [ ]:
df.sort_values("Jobs", ascending=False)[["Company","Jobs"]].head(10)

# Q11. Which Location Offers Highest Average Salary?

In [ ]:
df.groupby("Location")["Salaries"].mean().sort_values(ascending=False).head()

# Q12: Most Frequently Highly Rated Aspects ?

In [ ]:
df['Highly_Rated_For'].value_counts().head(5)


# Q13: Most Common Industries ?

In [ ]:
df['Industry'].value_counts().head(5)

# Q14: Most Frequently Criticized Aspects ?

In [ ]:
df['Critically_Rated_For'].value_counts().head(10)

In [ ]:
df['Company'].value_counts()

In [ ]:
df['Industry'].value_counts().head(5)

In [ ]:
df['Highly_Rated_For'].value_counts().head(5)

In [ ]:
df['Location'].value_counts().head(5)

In [ ]:
df['Critically_Rated_For'].value_counts().head(10)